In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.compose import ColumnTransformer

In [17]:
import pandas as pd
#Chargement de la base de données
df = pd.read_excel("D:/UADB/MEMOIRE/Data/GastricCancerData.xlsx")
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Identifier les colonnes catégoriques
CatCols = df.select_dtypes(include=['object']).columns
# Encodage des variables catégoriques
#Label Encoding
label_encoder = LabelEncoder()
for col in CatCols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

df.head()

,AGE,SEXE,Cardiopathie,Ulceregastrique,Douleurepigastrique,Ulcero-bourgeonnant,Constipation,Denitrution,Tabac,Mucineux,Tubuleux,Infiltrant,Stenosant,Metastases,Adenopathie,Traitement,Tempsdesuivi (Mois),Deces
0,65,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1
1,34,0,0,0,1,0,1,0,1,1,1,1,1,1,1,0,1,1
2,55,1,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,1
3,60,0,1,1,1,0,1,0,1,1,1,1,0,1,0,1,2,0
4,65,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,2,1


In [18]:
# Effacer les variables non pertinantes
clean=['SEXE','Tubuleux','Constipation']
data=df.drop(columns=clean,axis=1)
data.columns

Index(['AGE', 'Cardiopathie', 'Ulceregastrique', 'Douleurepigastrique',
       'Ulcero-bourgeonnant', 'Denitrution', 'Tabac', 'Mucineux', 'Infiltrant',
       'Stenosant', 'Metastases', 'Adenopathie', 'Traitement',
       'Tempsdesuivi (Mois)', 'Deces'],
      dtype='object')

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Supposons que df est votre DataFrame
# df = pd.read_csv("votre_data.csv")

# Variables explicatives
X = df.drop(["Deces", "Tempsdesuivi (Mois)", "Traitement"], axis=1)

# Variables cibles
y_deces = df['Deces']  # Décès à 5 ans (0 ou 1)
y_duree_survie = df['Tempsdesuivi (Mois)']  # Durée de survie en mois
y_reponse_traitement = df["Traitement"]  # Réponse au traitement (0 ou 1, hypothétique)

# Prétraitement (exemple) : mise à l'échelle des variables explicatives
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_deces_train, y_deces_test, y_duree_survie_train, y_duree_survie_test, y_reponse_traitement_train, y_reponse_traitement_test = train_test_split(
    X_scaled, y_deces, y_duree_survie, y_reponse_traitement, test_size=0.2, random_state=42)

# Définition de l'entrée
input_layer = Input(shape=(X_train.shape[1],))

# Couches partagées
shared_layer = Dense(64, activation='relu')(input_layer)
shared_layer = Dropout(0.3)(shared_layer)
shared_layer = Dense(32, activation='relu')(shared_layer)
shared_layer = Dropout(0.3)(shared_layer)

# Branche pour la prédiction du décès
deces_branch = Dense(16, activation='relu')(shared_layer)
deces_output = Dense(1, activation='sigmoid', name='Deces')(deces_branch)

# Branche pour la prédiction de la durée de survie
duree_survie_branch = Dense(16, activation='relu')(shared_layer)
duree_survie_output = Dense(1, activation='linear', name='Tempsdesuivi_Mois')(duree_survie_branch)

# Branche pour la prédiction de la réponse au traitement
reponse_traitement_branch = Dense(16, activation='relu')(shared_layer)
reponse_traitement_output = Dense(1, activation='sigmoid', name='Traitement')(reponse_traitement_branch)

# Création du modèle multitâches
model = Model(inputs=input_layer, outputs=[deces_output, duree_survie_output, reponse_traitement_output])

# Compilation du modèle avec MAE pour la durée de survie
model.compile(optimizer='adam',
              loss={'Deces': 'binary_crossentropy',
                    'Tempsdesuivi_Mois': 'mean_absolute_error',  # Utilisation de MAE
                    'Traitement': 'binary_crossentropy'},
              metrics={'Deces': 'accuracy',
                       'Tempsdesuivi_Mois': 'mae',
                       'Traitement': 'accuracy'})

# Affichage de l'architecture du modèle
model.summary()

# Entraînement du modèle
history = model.fit(X_train,
                    {'Deces': y_deces_train,
                     'Tempsdesuivi_Mois': y_duree_survie_train,
                     'Traitement': y_reponse_traitement_train},
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 15)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 64)                │           1,024 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_4 (Dropout)           │ (None, 64)                │               0 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 32)                │           2,080 │ dropout_4[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_5 (Dropout)           │ (None, 32)                │               0 │ dense_8[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 16)                │             528 │ dropout_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (None, 16)                │             528 │ dropout_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_11 (Dense)              │ (None, 16)                │             528 │ dropout_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deces (Dense)                 │ (None, 1)                 │              17 │ dense_9[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Tempsdesuivi_Mois (Dense)     │ (None, 1)                 │              17 │ dense_10[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Traitement (Dense)            │ (None, 1)                 │              17 │ dense_11[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 4,739 (18.51 KB)

 Trainable params: 4,739 (18.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 249ms/step - Deces_accuracy: 0.6134 - Deces_loss: 0.6913 - Tempsdesuivi_Mois_loss: 27.7843 - Tempsdesuivi_Mois_mae: 27.8001 - Traitement_accuracy: 0.5337 - Traitement_loss: 0.6969 - loss: 29.1881 - val_Deces_accuracy: 0.7037 - val_Deces_loss: 0.6743 - val_Tempsdesuivi_Mois_loss: 27.2068 - val_Tempsdesuivi_Mois_mae: 27.2506 - val_Traitement_accuracy: 0.4630 - val_Traitement_loss: 0.6934 - val_loss: 28.6123
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - Deces_accuracy: 0.6361 - Deces_loss: 0.6685 - Tempsdesuivi_Mois_loss: 27.6472 - Tempsdesuivi_Mois_mae: 27.6391 - Traitement_accuracy: 0.4587 - Traitement_loss: 0.7183 - loss: 29.0263 - val_Deces_accuracy: 0.7037 - val_Deces_loss: 0.6501 - val_Tempsdesuivi_Mois_loss: 26.8373 - val_Tempsdesuivi_Mois_mae: 26.8857 - val_Traitement_accuracy: 0.4630 - val_Traitement_loss: 0.6922 - val_loss: 28.2228
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - Deces_accuracy: 0.6210 - Deces_loss: 0.6828 - Tempsdes

In [20]:
# Affichage des résultats
print(f"Perte totale : {results[0]}")
print(f"Précision décès à 5 ans : {results[4]}")
print(f"MAE durée de survie : {results[5]}")
print(f"Précision réponse au traitement : {results[6]}")

# Prédictions
predictions = model.predict(X_test)
deces_pred, duree_survie_pred, reponse_traitement_pred = predictions

# Métriques supplémentaires
auc_deces = roc_auc_score(y_deces_test, deces_pred)
mae_duree_survie = mean_absolute_error(y_duree_survie_test, duree_survie_pred)
auc_reponse_traitement = roc_auc_score(y_reponse_traitement_test, reponse_traitement_pred)

print(f"AUC décès à 5 ans : {auc_deces}")
print(f"MAE durée de survie : {mae_duree_survie}")
print(f"AUC réponse au traitement : {auc_reponse_traitement}")

Perte totale : 478.76507568359375
Précision décès à 5 ans : 0.7352941036224365
MAE durée de survie : 477.416259765625
Précision réponse au traitement : 0.45588234066963196
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
AUC décès à 5 ans : 0.8766666666666667
MAE durée de survie : 10.463025093078613
AUC réponse au traitement : 0.3922705314009661


In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

# Supposons que votre DataFrame df contient les colonnes :
# - Variables explicatives (features) : toutes sauf "Deces", "Tempsdesuivi (Mois)" et "Traitement"
# - Cibles : "Deces", "Tempsdesuivi (Mois)" et "Traitement"

# Préparation des données
X = df.drop(["Deces", "Tempsdesuivi (Mois)", "Traitement"], axis=1)
y_deces = df['Deces'].values           # 1 = événement (décès), 0 = censuré
y_duree_survie = df['Tempsdesuivi (Mois)'].values  # Durée de suivi en mois
y_reponse_traitement = df["Traitement"].values     # Réponse au traitement (0 ou 1)

# Mise à l'échelle des variables explicatives
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Pour la branche survie, on combine la durée et l'événement dans un array à deux colonnes
y_survival = np.column_stack([y_duree_survie, y_deces])

# Division des données en ensembles d'entraînement et de test
# Pour la branche survie, la cible est y_survival
# Pour les autres branches, on utilise y_deces et y_reponse_traitement respectivement
X_train, X_test, y_survival_train, y_survival_test, y_reponse_traitement_train, y_reponse_traitement_test = train_test_split(
    X_scaled, y_survival, y_reponse_traitement, test_size=0.2, random_state=42)

# Pour la branche décès, on extrait l'indicateur d'événement de la cible survie
y_deces_train = y_survival_train[:, 1]
y_deces_test = y_survival_test[:, 1]

# -------------------------------
# Définition d'une fonction de perte personnalisée pour le modèle Cox
# -------------------------------
def cox_ph_loss(y_true, y_pred):
    """
    y_true : tenseur de forme (batch, 2) où y_true[:,0] = durée de survie, y_true[:,1] = événement (1 = décès, 0 = censuré)
    y_pred : score de risque (pas de transformation, activé linéairement)
    La fonction réalise un tri par durée décroissante et calcule la vraisemblance partielle.
    """
    # Extraction du temps et de l'événement
    time = y_true[:, 0]
    event = y_true[:, 1]
    
    # Tri décroissant selon le temps
    order = tf.argsort(time, direction='DESCENDING')
    time = tf.gather(time, order)
    event = tf.gather(event, order)
    risk = tf.gather(y_pred, order)
    
    # Calcul de la somme cumulée de exp(score de risque)
    exp_risk = tf.exp(risk)
    cum_sum = tf.cumsum(exp_risk)
    
    # Log-vraisemblance partielle
    log_likelihood = risk - tf.math.log(cum_sum)
    # Ne considérer que les événements observés
    uncensored_ll = log_likelihood * event
    # On cherche à maximiser la vraisemblance => on minimise l'opposé
    loss = -tf.reduce_mean(uncensored_ll)
    return loss

# -------------------------------
# Construction du modèle multi-tâches robuste
# -------------------------------

# Définition de l'entrée
input_layer = Input(shape=(X_train.shape[1],), name='Input')

# Bloc de couches partagées renforcé (plus de neurones et dropout pour éviter l'overfitting)
shared = Dense(128, activation='relu')(input_layer)
shared = Dropout(0.4)(shared)
shared = Dense(64, activation='relu')(shared)
shared = Dropout(0.4)(shared)
shared = Dense(32, activation='relu')(shared)

# Branche survie (prédit un score de risque utilisé dans la fonction de perte Cox)
survival_branch = Dense(16, activation='relu')(shared)
survival_output = Dense(1, activation='linear', name='Survival')(survival_branch)

# Branche décès (prédiction binaire)
death_branch = Dense(16, activation='relu')(shared)
death_output = Dense(1, activation='sigmoid', name='Death')(death_branch)

# Branche réponse au traitement (prédiction binaire)
treatment_branch = Dense(16, activation='relu')(shared)
treatment_output = Dense(1, activation='sigmoid', name='Treatment')(treatment_branch)

# Définition du modèle avec trois sorties
model = Model(inputs=input_layer, outputs=[death_output, survival_output, treatment_output])

# Compilation du modèle :
# - Pour la branche survie, on utilise notre loss personnalisé (Cox)
# - Pour les branches décès et traitement, on utilise la binary_crossentropy
model.compile(optimizer='adam',
              loss={'Death': 'binary_crossentropy',
                    'Survival': cox_ph_loss,
                    'Treatment': 'binary_crossentropy'},
              metrics={'Death': 'accuracy',
                       'Survival': 'mae',        # Note : MAE n'est pas forcément pertinent pour Cox, mais on le garde pour suivi
                       'Treatment': 'accuracy'})

model.summary()

# -------------------------------
# Entraînement du modèle
# -------------------------------
history = model.fit(X_train,
                    {'Death': y_deces_train,
                     'Survival': y_survival_train,
                     'Treatment': y_reponse_traitement_train},
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)

# -------------------------------
# Évaluation du modèle
# -------------------------------
results = model.evaluate(X_test,
                         {'Death': y_deces_test,
                          'Survival': y_survival_test,
                          'Treatment': y_reponse_traitement_test})
print("Résultats sur le jeu de test :", results)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)            │ (None, 15)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_12 (Dense)              │ (None, 128)               │           2,048 │ Input[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_6 (Dropout)           │ (None, 128)               │               0 │ dense_12[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_13 (Dense)              │ (None, 64)                │           8,256 │ dropout_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_7 (Dropout)           │ (None, 64)                │               0 │ dense_13[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_14 (Dense)              │ (None, 32)                │           2,080 │ dropout_7[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_16 (Dense)              │ (None, 16)                │             528 │ dense_14[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_15 (Dense)              │ (None, 16)                │             528 │ dense_14[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_17 (Dense)              │ (None, 16)                │             528 │ dense_14[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Death (Dense)                 │ (None, 1)                 │              17 │ dense_16[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Survival (Dense)              │ (None, 1)                 │              17 │ dense_15[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Treatment (Dense)             │ (None, 1)                 │              17 │ dense_17[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 14,019 (54.76 KB)

 Trainable params: 14,019 (54.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 176ms/step - Death_accuracy: 0.6615 - Death_loss: 0.6713 - Survival_loss: 1.7167 - Survival_mae: 15.4252 - Treatment_accuracy: 0.5103 - Treatment_loss: 0.7120 - loss: 3.1012 - val_Death_accuracy: 0.7037 - val_Death_loss: 0.6668 - val_Survival_loss: 1.6315 - val_Survival_mae: 14.4061 - val_Treatment_accuracy: 0.5370 - val_Treatment_loss: 0.6820 - val_loss: 3.0263
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - Death_accuracy: 0.5886 - Death_loss: 0.6721 - Survival_loss: 1.5644 - Survival_mae: 16.0538 - Treatment_accuracy: 0.5474 - Treatment_loss: 0.6908 - loss: 2.9261 - val_Death_accuracy: 0.7037 - val_Death_loss: 0.6547 - val_Survival_loss: 1.6132 - val_Survival_mae: 14.3943 - val_Treatment_accuracy: 0.5370 - val_Treatment_loss: 0.6845 - val_loss: 2.9960
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - Death_accuracy: 0.6290 - Death_loss: 0.6553 - Survival_loss: 1.5809 - Survival_mae: 15.5489 - Treatment_accuracy: 0.5048 - Treatment_loss: 

In [26]:
# Affichage des résultats
print(f"Perte totale : {results[0]:.2f}")
print(f"Précision décès à 5 ans : {results[4]:.2f}")
print(f"MAE durée de survie : {results[5]}")
print(f"Précision réponse au traitement : {results[6]:.2f}")

# Prédictions
predictions = model.predict(X_test)
deces_pred, duree_survie_pred, reponse_traitement_pred = predictions

# Métriques supplémentaires
auc_deces = roc_auc_score(y_deces_test, deces_pred)
mae_duree_survie = mean_absolute_error(y_duree_survie_test, duree_survie_pred)
auc_reponse_traitement = roc_auc_score(y_reponse_traitement_test, reponse_traitement_pred)

print(f"AUC décès à 5 ans : {auc_deces:.2F}")
print(f"MAE durée de survie : {mae_duree_survie:.2f}")
print(f"AUC réponse au traitement : {auc_reponse_traitement:.2}")

Perte totale : 2.52
Précision décès à 5 ans : 0.84
MAE durée de survie : 14.651822090148926
Précision réponse au traitement : 0.62
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
AUC décès à 5 ans : 0.87
MAE durée de survie : 27.88
AUC réponse au traitement : 0.64


In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored

# Variables cibles
T = df['Tempsdesuivi (Mois)'].values  # Temps de survie
E = df['Deces'].values  # Événement (1 = décès, 0 = censuré)
Y_treatment = df['Traitement'].values  # Réponse au traitement (binaire)

# Division des données
X_train, X_test, T_train, T_test, E_train, E_test, Y_treatment_train, Y_treatment_test = train_test_split(
    X, T, E, Y_treatment, test_size=0.2, random_state=42)

# Définition du modèle DeepHit
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)

# Sortie pour la prédiction du décès (classification binaire)
death_output = Dense(1, activation='sigmoid', name='Deces')(x)

# Sortie pour la prédiction du temps de survie (régression)
survival_output = Dense(1, activation='relu', name='Tempsdesuivi')(x)

# Sortie pour la réponse au traitement (classification binaire)
treatment_output = Dense(1, activation='sigmoid', name='Traitement')(x)

# Compilation du modèle
model = Model(inputs=input_layer, outputs=[death_output, survival_output, treatment_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss={'Deces': 'binary_crossentropy',
                    'Tempsdesuivi': 'mean_absolute_error',
                    'Traitement': 'binary_crossentropy'},
              metrics={'Deces': 'accuracy',
                       'Tempsdesuivi': 'mae',
                       'Traitement': 'accuracy'})

# Entraînement
history = model.fit(X_train,
                    {'Deces': E_train,
                     'Tempsdesuivi': T_train,
                     'Traitement': Y_treatment_train},
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)

# Prédictions
predictions = model.predict(X_test)
death_pred, survival_pred, treatment_pred = predictions

# Calcul du C-index pour la survie
c_index = concordance_index_censored(E_test.astype(bool), T_test, survival_pred.flatten())[0]
print(f"C-index survie : {c_index}")

# AUC pour les autres tâches
from sklearn.metrics import roc_auc_score, mean_absolute_error
auc_deces = roc_auc_score(E_test, death_pred)
auc_traitement = roc_auc_score(Y_treatment_test, treatment_pred)
mae_survie = mean_absolute_error(T_test, survival_pred)

print(f"AUC décès à 5 ans : {auc_deces}")
print(f"MAE durée de survie : {mae_survie}")
print(f"AUC réponse au traitement : {auc_traitement}")


Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 228ms/step - Deces_accuracy: 0.4268 - Deces_loss: 0.9703 - Tempsdesuivi_loss: 30.1890 - Tempsdesuivi_mae: 30.2171 - Traitement_accuracy: 0.5554 - Traitement_loss: 0.7679 - loss: 33.0709 - val_Deces_accuracy: 0.2963 - val_Deces_loss: 0.9254 - val_Tempsdesuivi_loss: 27.6875 - val_Tempsdesuivi_mae: 27.7222 - val_Traitement_accuracy: 0.4630 - val_Traitement_loss: 0.7849 - val_loss: 30.4938
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - Deces_accuracy: 0.4029 - Deces_loss: 0.9498 - Tempsdesuivi_loss: 27.1542 - Tempsdesuivi_mae: 27.1308 - Traitement_accuracy: 0.4673 - Traitement_loss: 0.7927 - loss: 29.9217 - val_Deces_accuracy: 0.2963 - val_Deces_loss: 0.8767 - val_Tempsdesuivi_loss: 27.6875 - val_Tempsdesuivi_mae: 27.7222 - val_Traitement_accuracy: 0.4630 - val_Traitement_loss: 1.6157 - val_loss: 31.1806
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - Deces_accuracy: 0.4810 - Deces_loss: 0.8168 - Tempsdesuivi_loss: 30.7278 - Tempsdesuivi_mae

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored
from sklearn.metrics import roc_auc_score, mean_absolute_error


# Variables cibles
T = np.log(df['Tempsdesuivi (Mois)'].values + 1)  # Transformation log(T+1)
E = df['Deces'].values  # Événement (1 = décès, 0 = censuré)
Y_treatment = df['Traitement'].values  # Réponse au traitement (binaire)

# Division des données
X_train, X_test, T_train, T_test, E_train, E_test, Y_treatment_train, Y_treatment_test = train_test_split(
    X, T, E, Y_treatment, test_size=0.2, random_state=42)

# Définition de la fonction de perte de Cox (log-partial likelihood)
def cox_loss(y_true, y_pred):
    event, time = y_true[:, 0], y_true[:, 1]
    risk = tf.exp(y_pred)  # Score de risque exponentié
    log_risk = tf.math.log(tf.cumsum(risk[::-1])[::-1] + 1e-9)
    loss = -tf.reduce_mean(event * (y_pred - log_risk))  # Log-partial likelihood
    return loss

# Création du modèle
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(256, kernel_regularizer=l2(0.01))(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)

x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)

# Sortie pour la prédiction du décès (classification binaire)
death_output = Dense(1, activation='sigmoid', name='Deces')(x)

# Sortie pour la prédiction du temps de survie (régression cox)
survival_output = Dense(1, activation='linear', name='Tempsdesuivi')(x)

# Sortie pour la réponse au traitement (classification binaire)
treatment_output = Dense(1, activation='sigmoid', name='Traitement')(x)

# Compilation du modèle avec pondération dynamique des pertes
model = Model(inputs=input_layer, outputs=[death_output, survival_output, treatment_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss={'Deces': 'binary_crossentropy',
                    'Tempsdesuivi': cox_loss,
                    'Traitement': 'binary_crossentropy'},
              loss_weights={'Deces': 1.0, 'Tempsdesuivi': 2.0, 'Traitement': 1.0},  # Survie plus pondérée
              metrics={'Deces': 'accuracy', 'Traitement': 'accuracy'})

# Préparer les labels pour la survie
Y_train_survival = np.vstack([E_train, T_train]).T
Y_test_survival = np.vstack([E_test, T_test]).T

# Entraînement
history = model.fit(X_train,
                    {'Deces': E_train,
                     'Tempsdesuivi': Y_train_survival,
                     'Traitement': Y_treatment_train},
                    epochs=150,
                    batch_size=32,
                    validation_split=0.2)

# Prédictions
predictions = model.predict(X_test)
death_pred, survival_pred, treatment_pred = predictions

# Calcul du C-index pour la survie
c_index = concordance_index_censored(E_test.astype(bool), T_test, survival_pred.flatten())[0]
print(f"C-index survie : {c_index}")

# AUC pour les autres tâches
auc_deces = roc_auc_score(E_test, death_pred)
auc_traitement = roc_auc_score(Y_treatment_test, treatment_pred)
mae_survie = mean_absolute_error(T_test, survival_pred)

print(f"AUC décès à 5 ans : {auc_deces}")
print(f"MAE durée de survie : {mae_survie}")
print(f"AUC réponse au traitement : {auc_traitement}")


Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 209ms/step - Deces_accuracy: 0.4897 - Deces_loss: 0.8869 - Tempsdesuivi_loss: 2.0801 - Traitement_accuracy: 0.4947 - Traitement_loss: 0.9030 - loss: 8.8050 - val_Deces_accuracy: 0.2963 - val_Deces_loss: 1.4045 - val_Tempsdesuivi_loss: 1.6678 - val_Traitement_accuracy: 0.4630 - val_Traitement_loss: 2.9805 - val_loss: 10.5826
Epoch 2/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - Deces_accuracy: 0.5197 - Deces_loss: 0.8009 - Tempsdesuivi_loss: 1.7890 - Traitement_accuracy: 0.4550 - Traitement_loss: 0.8932 - loss: 8.1023 - val_Deces_accuracy: 0.2963 - val_Deces_loss: 1.9631 - val_Tempsdesuivi_loss: 1.8125 - val_Traitement_accuracy: 0.4630 - val_Traitement_loss: 2.8364 - val_loss: 11.2483
Epoch 3/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - Deces_accuracy: 0.6028 - Deces_loss: 0.7026 - Tempsdesuivi_loss: 1.7396 - Traitement_accuracy: 0.5031 - Traitement_loss: 0.7929 - loss: 7.7748 - val_Deces_accuracy: 0.2963 - val_Deces_loss: 1.5788 - val_Tempsdesuivi_